In [ ]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv("G:\VSCODE\Python And Data Sceince\Advance AI Projects\\aio protype system\Data\\all_content_with_posters.csv")

<>:1: SyntaxWarning: invalid escape sequence '\V'
<>:1: SyntaxWarning: invalid escape sequence '\V'
C:\Users\sharm\AppData\Local\Temp\ipykernel_1840\4029382526.py:1: SyntaxWarning: invalid escape sequence '\V'
  df = pd.read_csv("G:\VSCODE\Python And Data Sceince\Advance AI Projects\\aio protype system\Data\\all_content_with_posters.csv")


In [12]:
df['name'] = df['name'].astype(str).str.lower()
df['genres'] = df['genres'].astype(str).str.lower()
df['type'] = df['type'].astype(str).str.lower()


In [13]:
def classify(row):
    g = row['genres']
    t = row['type']
    
    # 1️⃣ Animation goes directly to anime
    if "animation" in g:
        return "anime"
    
    # 2️⃣ Live action movies
    if t == "movie" or row.get('runtime', 0) > 70:
        return "movie"
    
    # 3️⃣ Everything else is a webseries
    return "webseries"

df['clean_type'] = df.apply(classify, axis=1)

df['clean_type'].value_counts()

clean_type
movie        2812
anime        1729
webseries    1309
Name: count, dtype: int64

In [15]:
df.to_csv("cleaned_dataset_version2.csv", index=False)

In [14]:
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize


In [16]:

df = pd.read_csv("cleaned_dataset_version2.csv")

In [17]:
df['name'] = df['name'].astype(str).str.lower()
df['genres'] = df['genres'].astype(str).str.lower().fillna("")
df['overview'] = df['overview'].astype(str).str.lower().fillna("")
df['meta_text'] = df['meta_text'].astype(str).str.lower().fillna("")
df['clean_type'] = df['clean_type'].astype(str).str.lower()

In [18]:
title_to_idx = {title: i for i, title in enumerate(df['name'])}

len(title_to_idx), list(title_to_idx.items())[:5]

(5519,
 [('frankenstein', 2146),
  ('playdate', 1),
  ('predator badlands', 2),
  ('code 3', 3),
  ('black phone 2', 4)])

In [19]:
tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.8,
    ngram_range=(1,2),
    stop_words='english'
)

X_tfidf = tfidf.fit_transform(df['meta_text'])
X_tfidf.shape


(5850, 22358)

In [20]:
tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.8,
    ngram_range=(1,2),
    stop_words='english'
)

X_tfidf = tfidf.fit_transform(df['meta_text'])
X_tfidf.shape


(5850, 22358)

In [22]:
title_vec = CountVectorizer(
    min_df=2,
    max_df=0.9,
    ngram_range=(1,2),
    stop_words='english'
)

X_title = title_vec.fit_transform(df['name'])
X_title.shape


(5850, 2385)

In [23]:
genre_vec = CountVectorizer(token_pattern=r'[^,]+')  # split on commas
X_genre = genre_vec.fit_transform(df['genres'])
X_genre.shape


(5850, 55)

In [24]:
from sentence_transformers import SentenceTransformer

# compact & strong model
sbert_model = SentenceTransformer("all-mpnet-base-v2")

# this may take a few minutes the first time
embeddings = sbert_model.encode(
    df['meta_text'].tolist(),
    batch_size=64,
    show_progress_bar=True,
)
embeddings = np.array(embeddings)
embeddings.shape


c:\Users\sharm\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 92/92 [09:10<00:00,  5.98s/it]


(5850, 768)

In [25]:
base_path = r"G:\VSCODE\Python And Data Sceince\Advance AI Projects\\aio protype system\\models_version2"

# meta records (df + title map)
meta_records = {
    "meta_df": df,
    "title_to_idx": title_to_idx
}
joblib.dump(meta_records, base_path + r"\meta_records.joblib")

# tfidf + matrix
joblib.dump(tfidf, base_path + r"\tfidf_vectorizer.joblib")
joblib.dump(X_tfidf, base_path + r"\X_tfidf.joblib")

# title count vec + matrix
joblib.dump(title_vec, base_path + r"\count_vectorizer.joblib")
joblib.dump(X_title, base_path + r"\X_count.joblib")

# genre vec + matrix (NEW)
joblib.dump(genre_vec, base_path + r"\genre_vectorizer.joblib")
joblib.dump(X_genre, base_path + r"\X_genre.joblib")

# SBERT embeddings
joblib.dump(embeddings, base_path + r"\sbert_embeddings.joblib")

print("All artifacts saved ✅")


All artifacts saved ✅
